# Helpdesk Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

 ····


In [2]:
shift = pd.read_sql_table('Shift', engine)
staff = pd.read_sql_table('Staff', engine)
issue = pd.read_sql_table('Issue', engine)
shift_type = pd.read_sql_table('Shift_type', engine)
level = pd.read_sql_table('Level', engine)
customer = pd.read_sql_table('Customer', engine)
caller = pd.read_sql_table('Caller', engine)

## 6.
List the Company name and the number of calls for those companies with more than 18 calls.

```
+------------------+----+
| Company_name     | cc |
+------------------+----+
| Gimmick Inc.     | 22 |
| Hamming Services | 19 |
| High and Co.     | 20 |
+------------------+----+
```

In [3]:
(issue.merge(caller, on='Caller_id')
 .merge(customer, on='Company_ref')
 [['Company_name', 'Call_ref']]
 .groupby('Company_name')
 .count().reset_index()
 .query('Call_ref>18'))

,Company_name,Call_ref
16,Gimmick Inc.,22
18,Hamming Services,19
20,High and Co.,20


## 7.
Find the callers who have never made a call. Show first name and last name

```
+------------+-----------+
| first_name | last_name |
+------------+-----------+
| David      | Jackson   |
| Ethan      | Phillips  |
+------------+-----------+
```

In [4]:
(caller[~caller['Caller_id'].isin(issue['Caller_id'])]
 [['First_name', 'Last_name']])

,First_name,Last_name
10,David,Jackson
126,Ethan,Phillips


## 8.
For each customer show: Company name, contact name, number of calls where the number of calls is fewer than 5

```
+--------------------+------------+-----------+----+
| Company_name       | first_name | last_name | nc |
+--------------------+------------+-----------+----+
| Pitiable Shipping  | Ethan      | McConnell |  4 |
| Rajab Group        | Emily      | Cooper    |  4 |
| Somebody Logistics | Ethan      | Phillips  |  2 |
+--------------------+------------+-----------+----+
```

In [5]:
(issue.merge(caller[['Caller_id', 'Company_ref']], on='Caller_id')
 .merge(customer, on='Company_ref')
 .merge(caller[['Caller_id', 'First_name', 'Last_name']], 
        left_on='Contact_id', right_on='Caller_id')
 [['Company_ref', 'Company_name', 'First_name', 'Last_name', 'Call_ref']]
 .groupby(['Company_ref', 'Company_name'])
 .count().reset_index()
 .query('Call_ref<5')
 [['Company_name', 'First_name', 'Last_name', 'Call_ref']])

,Company_name,First_name,Last_name,Call_ref
4,Somebody Logistics,2,2,2
28,Pitiable Shipping,4,4,4
43,Rajab Group,4,4,4


## 9.
For each shift show the number of staff assigned. Beware that some roles may be NULL and that the same person might have been assigned to multiple roles (The roles are 'Manager', 'Operator', 'Engineer1', 'Engineer2').

```
+------------+------------+----+
| Shift_date | Shift_type | cw |
+------------+------------+----+
| 2017-08-12 | Early      |  4 |
| 2017-08-12 | Late       |  4 |
| 2017-08-13 | Early      |  3 |
| 2017-08-13 | Late       |  2 |
| 2017-08-14 | Early      |  4 |
| 2017-08-14 | Late       |  4 |
| 2017-08-15 | Early      |  4 |
| 2017-08-15 | Late       |  4 |
| 2017-08-16 | Early      |  4 |
| 2017-08-16 | Late       |  4 |
+------------+------------+----+
```

In [6]:
# SELECT agg."Shift_date", agg."Shift_type", COUNT(agg.p) cw FROM
# (SELECT DISTINCT a."Shift_date", a."Shift_type", a.p FROM
#   (SELECT "Shift_date", "Shift_type", "Manager" p FROM "Shift" 
#     UNION SELECT "Shift_date", "Shift_type", "Operator" p FROM "Shift" 
#     UNION SELECT "Shift_date", "Shift_type", "Engineer1" p FROM "Shift" 
#     UNION SELECT "Shift_date", "Shift_type", "Engineer2" p FROM "Shift" 
#   ) a
# ) AS agg
# GROUP BY agg."Shift_date", agg."Shift_type"
# ORDER BY agg."Shift_date", agg."Shift_type";

(shift[['Shift_date', 'Shift_type', 'Manager', 
        'Operator', 'Engineer1', 'Engineer2']]
 .set_index(['Shift_date', 'Shift_type'])
 .stack().reset_index()
 .rename(columns={'level_2': 'role', 0: 'p'})
 .drop_duplicates(subset=['Shift_date', 'Shift_type', 'p'])
 [['Shift_date', 'Shift_type', 'role']]
 .groupby(['Shift_date', 'Shift_type'])
 .count().reset_index())

,Shift_date,Shift_type,role
0,2017-08-12,Early,4
1,2017-08-12,Late,4
2,2017-08-13,Early,3
3,2017-08-13,Late,2
4,2017-08-14,Early,4
5,2017-08-14,Late,4
6,2017-08-15,Early,4
7,2017-08-15,Late,4
8,2017-08-16,Early,4
9,2017-08-16,Late,4


## 10.
Caller 'Harry' claims that the operator who took his most recent call was abusive and insulting. Find out who took the call (full name) and when.

```
+------------+-----------+---------------------+
| first_name | last_name | call_date           |
+------------+-----------+---------------------+
| Emily      | Best      | 2017-08-16 10:25:00 |
+------------+-----------+---------------------+
```

In [7]:
(caller.loc[caller['First_name']=='Harry', ['Caller_id']]
 .merge(issue[['Taken_by', 'Caller_id', 'Call_date']], on='Caller_id')
 .merge(staff, left_on='Taken_by', right_on='Staff_code')
 [['First_name', 'Last_name', 'Call_date']]
 .sort_values('Call_date', ascending=False)
 .head(1))

,First_name,Last_name,Call_date
3,Emily,Best,2017-08-16 10:25:00
